Connected to Python 3.11.4

In [2]:
from bs4 import BeautifulSoup, element
import urllib
import urllib.request
import pandas as pd
import numpy as np
import os

In [12]:
pages = 3
rec_count = 0
rank = []
gname = []
platform = []
year = []
genre = []
critic_score = []
user_score = []
publisher = []
developer = []
sales_na = []
sales_pal = []
sales_jp = []
sales_ot = []
sales_gl = []

urlhead = 'http://www.vgchartz.com/gamedb/?page='
urltail = '&console=&region=All&developer=&publisher=&genre=&boxart=Both&ownership=Both'
urltail += '&results=25&order=Sales&showtotalsales=0&showtotalsales=1&showpublisher=0'
urltail += '&showpublisher=1&showvgchartzscore=0&shownasales=1&showdeveloper=1&showcriticscore=1'
urltail += '&showpalsales=0&showpalsales=1&showreleasedate=1&showuserscore=1&showjapansales=1'
urltail += '&showlastupdate=0&showothersales=1&showgenre=1&sort=GL'

for page in range(1, pages):
    surl = urlhead + str(page) + urltail
    # urllib error, no att request
    r = urllib.request.urlopen(surl, data = None).read()
    # add features = html.parser
    soup = BeautifulSoup(r,features = "html.parser" )
    print(f"Page: {page}")

    # vgchartz website is really weird so we have to search for
    # <a> tags with game urls
    #game_tags = list(filter(
        #lambda x: x.attrs['href'].startswith('http://www.vgchartz.com/game/'),
        # discard the first 10 elements because those
        # links are in the navigation bar
        #soup.find_all("a")
    # '' Pull request quick fix
    game_tags = list(filter(
        lambda x: 'href' in x.attrs and x.attrs['href'].startswith('https://www.vgchartz.com/game/'),
        # discard the first 10 elements because those
        # links are in the navigation bar
        soup.find_all("a")
    ))[10:]

    for tag in game_tags:
        
        # add name to list
        gname.append(" ".join(tag.string.split()))
        print(f"{rec_count + 1} Fetch data for game {gname[-1]}")

        # get different attributes
        # traverse up the DOM tree
        data = tag.parent.parent.find_all("td")
        rank.append(np.int32(data[0].string))
        platform.append(data[3].find('img').attrs['alt'])
        publisher.append(data[4].string)
        developer.append(data[5].string)
        critic_score.append(
            float(data[6].string) if
            not data[6].string.startswith("N/A") else np.nan)
        user_score.append(
            float(data[7].string) if
            not data[7].string.startswith("N/A") else np.nan)
        sales_na.append(
            float(data[9].string[:-1]) if
            not data[9].string.startswith("N/A") else np.nan)
        sales_pal.append(
            float(data[10].string[:-1]) if
            not data[10].string.startswith("N/A") else np.nan)
        sales_jp.append(
            float(data[11].string[:-1]) if
            not data[11].string.startswith("N/A") else np.nan)
        sales_ot.append(
            float(data[12].string[:-1]) if
            not data[12].string.startswith("N/A") else np.nan)
        sales_gl.append(
            float(data[8].string[:-1]) if
            not data[8].string.startswith("N/A") else np.nan)
        release_year = data[13].string.split()[-1]
        # different format for year
        if release_year.startswith('N/A'):
            year.append('N/A')
        else:
            if int(release_year) >= 80:
                year_to_add = np.int32("19" + release_year)
            else:
                year_to_add = np.int32("20" + release_year)
            year.append(year_to_add)

        # go to every individual website to get genre info
        url_to_game = tag.attrs['href']
        site_raw = urllib.request.urlopen(url_to_game).read()
        sub_soup = BeautifulSoup(site_raw, "html.parser")
        # again, the info box is inconsistent among games so we
        # have to find all the h2 and traverse from that to the genre name
        h2s = sub_soup.find("div", {"id": "gameGenInfoBox"}).find_all('h2')
        # make a temporary tag here to search for the one that contains
        # the word "Genre"
        temp_tag = element.Tag
        for h2 in h2s:
            if h2.string == 'Genre':
                temp_tag = h2
        genre.append(temp_tag.next_sibling.string)

        rec_count += 1

columns = {
    'Rank': rank,
    'Name': gname,
    'Platform': platform,
    'Year': year,
    'Genre': genre,
    'Critic_Score': critic_score,
    'User_Score': user_score,
    'Publisher': publisher,
    'Developer': developer,
    'NA_Sales': sales_na,
    'PAL_Sales': sales_pal,
    'JP_Sales': sales_jp,
    'Other_Sales': sales_ot,
    'Global_Sales': sales_gl
}

Page: 1
1 Fetch data for game The Sims
2 Fetch data for game Final Fantasy
3 Fetch data for game Grand Theft Auto V
4 Fetch data for game Mario Kart
5 Fetch data for game Need for Speed
6 Fetch data for game Resident Evil
7 Fetch data for game Tomb Raider
8 Fetch data for game The Legend of Zelda
9 Fetch data for game NBA 2K
10 Fetch data for game Madden NFL
11 Fetch data for game Wii/Nintendo Sports
12 Fetch data for game Pro Evolution Soccer/eFootball
13 Fetch data for game Sonic the Hedgehog
14 Fetch data for game Monster Hunter
15 Fetch data for game The Elder Scrolls
Page: 2
16 Fetch data for game The Witcher
17 Fetch data for game Worms
18 Fetch data for game Super Smash Bros.
19 Fetch data for game Mario Party
20 Fetch data for game Just Dance
21 Fetch data for game PlayerUnknown's Battlegrounds
22 Fetch data for game WWE 2K
23 Fetch data for game Dragon Ball
24 Fetch data for game Civilization
25 Fetch data for game God of War
26 Fetch data for game Counter-Strike
27 Fetch data

In [13]:
print(game_tags)

[<a href="https://www.vgchartz.com/game/226111/the-witcher/?region=All">The Witcher    </a>, <a href="https://www.vgchartz.com/game/228118/worms/?region=All">Worms    </a>, <a href="https://www.vgchartz.com/game/226193/super-smash-bros/?region=All">Super Smash Bros.    </a>, <a href="https://www.vgchartz.com/game/226170/mario-party/?region=All">Mario Party    </a>, <a href="https://www.vgchartz.com/game/226101/just-dance/?region=All">Just Dance    </a>, <a href="https://www.vgchartz.com/game/227625/playerunknowns-battlegrounds/?region=All">PlayerUnknown's Battlegrounds    </a>, <a href="https://www.vgchartz.com/game/226269/wwe-2k/?region=All">WWE 2K    </a>, <a href="https://www.vgchartz.com/game/226286/dragon-ball/?region=All">Dragon Ball    </a>, <a href="https://www.vgchartz.com/game/226068/civilization/?region=All">Civilization    </a>, <a href="https://www.vgchartz.com/game/226282/god-of-war/?region=All">God of War    </a>, <a href="https://www.vgchartz.com/game/226175/counter-str

In [10]:
print(rec_count)
df = pd.DataFrame(columns)
print(df.columns)
df = df[[
    'Rank', 'Name', 'Platform', 'Year', 'Genre',
    'Publisher', 'Developer', 'Critic_Score', 'User_Score',
    'NA_Sales', 'PAL_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']]

10
Index(['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Critic_Score',
       'User_Score', 'Publisher', 'Developer', 'NA_Sales', 'PAL_Sales',
       'JP_Sales', 'Other_Sales', 'Global_Sales'],
      dtype='object')


In [11]:
print(df)

   Rank                            Name Platform  Year             Genre  \
0    21             Wii/Nintendo Sports   Series  2006            Sports   
1    22  Pro Evolution Soccer/eFootball   Series  1995            Sports   
2    23              Sonic the Hedgehog   Series  1991          Platform   
3    24                  Monster Hunter   Series  2004      Role-Playing   
4    25               The Elder Scrolls   Series  1994      Role-Playing   
5    46                  Counter-Strike   Series  2000           Shooter   
6    47                     Donkey Kong   Series  1981          Platform   
7    48                The Oregon Trail   Series  2071         Education   
8    49     The Elder Scrolls V: Skyrim      All  2011      Role-Playing   
9    50                      Metal Gear   Series  1987  Action-Adventure   

              Publisher                Developer  Critic_Score  User_Score  \
0            Nintendo                 Nintendo             NaN         NaN   
1      

In [8]:
# change the name of the file
df.to_csv(os.getcwd()+r"\vgsales7.24.23.csv", sep=",", encoding='utf-8', index=False)